In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
import joblib
import json
from sqlalchemy import create_engine 
from datetime import datetime, date
import sys
pd.set_option('display.max_columns', None)

### Get Closest Date for each IBVStatusID

In [6]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER

drop table if EXISTS #t1
select A.Application_ID, A.PortfolioID, A.CustomerSSN, A.CustomerID, A.ApplicationDate, CustEmail,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') and A.ApplicationStatus='J'  then 1 else 0 end as PartialPreApproved,
case when L.LoanStatus not in ('V','W','G','K')  then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name, OriginationDate, LOANSTATUS,
case when ((L.RenewalLoanId <> '0') or (A.LPCampaign = 'RENEW'))  then 'RENEWAL' when ApplicationSteps not like '%R%' then 'NEW' else 'REPEAT' end as CustType, RenewalLoanId
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.AccountNumber = A.CustomerSSN
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.ApplicationID = A.Application_ID AND ibvT.PortfolioID= A.PortfolioID
where A.ApplicationDate>'2023-04-30' and ((CustEmail is null) or  (CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))

UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID
"""
query2 = """
WITH app_ibv AS (
  SELECT
    t.Application_ID,
    t.PortfolioID,
    t.ApplicationDate,
    ibv.IBVStatusID
  FROM #t1 t
  LEFT JOIN [LF_BankData].[dbo].[IBVStatus] ibv
    ON ibv.AccountNumber = t.CustomerSSN
   AND ibv.PortfolioID = t.PortfolioID
),
scored AS (
  SELECT
    cl.IBVStatusID,
    cl.DateProcessed,
    a.Application_ID,
    a.PortfolioID,
    a.ApplicationDate,
    ROW_NUMBER() OVER (
      PARTITION BY cl.IBVStatusID
      ORDER BY
        CASE WHEN a.ApplicationDate IS NULL THEN 1 ELSE 0 END,
        ABS(DATEDIFF_BIG(SECOND, a.ApplicationDate, cl.DateProcessed)),
        cl.DateProcessed DESC
    ) AS rn
  FROM [LF_LMS_Logs].[dbo].[CModelLogs] cl
  LEFT JOIN app_ibv a
    ON a.IBVStatusID = cl.IBVStatusID
  WHERE cl.ModelName = 'ibv'
)
SELECT *
FROM scored
WHERE rn = 1;
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_ibvIDs = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')



#### Join with the IBVStautsID Dedup first and then rerun the bankuity to see if there are improvements

In [7]:
print(df_ibvIDs.head(3))
print(df_ibvIDs.shape)

   IBVStatusID           DateProcessed  Application_ID  PortfolioID  \
0        36129 2025-12-04 10:47:45.630         93899.0          1.0   
1        36161 2025-11-03 12:17:41.490        126404.0          1.0   
2        36167 2025-12-04 10:35:09.087         91571.0          1.0   

          ApplicationDate  rn  
0 2025-07-11 11:55:48.000   1  
1 2025-11-03 02:19:18.000   1  
2 2025-07-04 01:13:54.217   1  
(12893, 6)


In [8]:
total_rows = len(df_ibvIDs)
unique_ids = df_ibvIDs['IBVStatusID'].nunique(dropna=True)
duplicate_count = total_rows - unique_ids
print(f"Total rows: {total_rows}")
print(f"Unique IBVStatusID: {unique_ids}")
print(f"Duplicate count: {duplicate_count}")

Total rows: 12893
Unique IBVStatusID: 12893
Duplicate count: 0


In [12]:
required_cols = {'IBVStatusID', 'DateProcessed'}
missing = required_cols - set(df_ibvIDs.columns)
assert not missing, f"df_ibvIDs missing columns: {missing}"

# Keep only required columns, normalize types, drop null IDs
_df = df_ibvIDs[['IBVStatusID', 'DateProcessed']].copy()
_df['IBVStatusID'] = pd.to_numeric(_df['IBVStatusID'], errors='coerce').astype('Int64')
_df['DateProcessed'] = pd.to_datetime(_df['DateProcessed'], errors='coerce')

# Final dataframe for rerun CSV
# - DateCreated column name is what rerun_model.py looks for
# - We keep datetime (CSV will serialize); rerun_model.py parses and uses YYYY-MM-DD

df_ibvIDs_for_csv = (
    _df.dropna(subset=['IBVStatusID'])
       .rename(columns={'DateProcessed': 'DateCreated'})
       .reset_index(drop=True)
)

df_ibvIDs_for_csv.head()

,IBVStatusID,DateCreated
0,36129,2025-12-04 10:47:45.630
1,36161,2025-11-03 12:17:41.490
2,36167,2025-12-04 10:35:09.087
3,36232,2025-12-04 13:21:27.203
4,36280,2025-12-22 15:41:40.907


#### Export the IBVStatusIDs + DateCreated for the Bankuity Rerun

In [13]:
import os

out_csv = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV6_dedup.csv"
os.makedirs(os.path.dirname(out_csv), exist_ok=True)

df_ibvIDs_for_csv.to_csv(out_csv, index=False)
out_csv


'/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV6_dedup.csv'